# Notebook summarizing PPMI dataset executions on VIP platform

In [83]:
import pandas as pd
import numpy as np
import json
import plotly.express as px
import plotly.colors as pcolors
import glob
import os
import re
from IPython.display import display
import joblib
from pathlib import Path

anonymizer = True

root_dir = Path.cwd()


def anondir(path: Path, prefix=root_dir) -> Path:
    """Anonymize a directory path by replacing user-specific parts with <root>."""
    if not anonymizer:
        return path
    path_str = str(path).replace(str(prefix), "<living-park>")
    return Path(path_str)


print(f"Root directory: {anondir(root_dir)}")
output_dir = root_dir / "cohort"
output_dir.mkdir(exist_ok=True)
print(f"Output directory: {anondir(output_dir)}")

Root directory: <living-park>
Output directory: <living-park>/cohort


## Cross-sectional

### List all executions for each subject and visit

In [84]:
# Get the list of runs (dataset + visit) present in each MCA repetition

mca_repetitions = glob.glob("vip_outputs/rep*/sub-*")
mca_repetitions = [
    execution
    for execution in mca_repetitions
    if ("long" not in execution and "base" not in execution)
]

df_vip = pd.DataFrame([], columns=["subject_visit", "repetition", "path"])

mca_repetitions_list = [
    {
        "subject_visit": os.path.basename(execution),
        "repetition": os.path.basename(os.path.dirname(execution)),
        "path": os.path.realpath(execution),
    }
    for execution in mca_repetitions
]

df_vip = pd.DataFrame(mca_repetitions_list)

# Associate a unique identifier to each subject_visit
patno_id = {sv: i for i, sv in enumerate(df_vip["subject_visit"].unique())}
df_vip["ID"] = df_vip["subject_visit"].apply(lambda x: patno_id[x])
df_vip["repetition"] = df_vip["repetition"].apply(
    lambda x: int(re.findall(r"\d+", x)[0])
)
df_vip["subject"] = df_vip["subject_visit"].apply(lambda x: x.split("_")[0])
df_vip["visit"] = df_vip["subject_visit"].apply(lambda x: x.split("_")[1])

filename = output_dir / "vip_executions_stats_info.csv"
df_vip.to_csv(filename, index=False)
print(f"Saved to {anondir(filename)}")

Saved to <living-park>/cohort/vip_executions_stats_info.csv


### Filter-out extra visits to keep 2 visits per subject

In [85]:
def get_subjects_with_n_visits(df, n_visits):
    """
    Get subjects with a specific number of visits.
    """
    subjects_count = df.groupby("subject")["visit"].nunique()
    subjects_with_n_visits = subjects_count[subjects_count == n_visits].index.tolist()
    print(f"Number of subjects with {n_visits} visits: {len(subjects_with_n_visits)}")
    return subjects_with_n_visits

In [86]:
subjects_with_1_visit = get_subjects_with_n_visits(df_vip, 1)
subjects_with_2_visits = get_subjects_with_n_visits(df_vip, 2)
subjects_with_3_visits = get_subjects_with_n_visits(df_vip, 3)

Number of subjects with 1 visits: 0
Number of subjects with 2 visits: 306
Number of subjects with 3 visits: 11


In [87]:
df_vip_2 = df_vip[df_vip["subject"].isin(subjects_with_2_visits)]
df_vip_3 = df_vip[df_vip["subject"].isin(subjects_with_3_visits)]
df_vip_3.sort_values(by=["subject", "visit"], inplace=True)
for subject in subjects_with_3_visits:
    df_subject = df_vip_3[df_vip_3["subject"] == subject]
    visits = df_subject["visit"].unique()
    if len(visits) > 2:
        visits_to_keep = visits[:2]
        df_vip_3 = df_vip_3[
            ~(
                (df_vip_3["subject"] == subject)
                & (~df_vip_3["visit"].isin(visits_to_keep))
            )
        ]
assert df_vip_3.groupby("subject")["visit"].nunique().max() == 2
df_vip = pd.concat([df_vip_2, df_vip_3])
df_vip.reset_index(drop=True, inplace=True)
assert df_vip.groupby("subject")["visit"].nunique().max() == 2

/tmp/ipykernel_209157/1698268813.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



## Ensure each subject has 2 visits for each repetition

In [88]:
import pandas as pd

df = df_vip.copy()
# pivot again, but inspect column names
pairs = df.pivot_table(
    index=["subject", "repetition"],
    columns="visit",
    values="subject_visit",
    aggfunc="first",
    observed=True,
).reset_index()

df_vip = pairs[(~pairs[pairs.columns[2:]].isnull()).sum(axis=1) == 2]
df_vip = df_vip.melt(
    id_vars=["subject", "repetition"],
    value_vars=df_vip.columns[2:],
    var_name="visit",
    value_name="subject_visit",
)

filename = output_dir / "vip_executions_stats_info_2visits.csv"
df_vip.to_csv(filename, index=False)
print(f"Saved to {anondir(filename)}")

Saved to <living-park>/cohort/vip_executions_stats_info_2visits.csv


In [89]:
unique_visit = df_vip.subject_visit.nunique()
print(f"Number of unique subject-visits in VIP outputs: {unique_visit}")

n_subjects_visits = df_vip[["subject", "visit"]]
n_subjects_visits["subject_visit"] = (
    n_subjects_visits["subject"] + "_" + n_subjects_visits["visit"]
)
# The number of occurrences of each subject_visit
n_subjects_visits = n_subjects_visits.groupby("subject_visit").size()
n_subjects_visits = n_subjects_visits.reset_index()
n_subjects_visits.columns = ["subject_visit", "count"]
n_subjects_visits = n_subjects_visits.sort_values(by="count", ascending=False)

Number of unique subject-visits in VIP outputs: 634


/tmp/ipykernel_209157/1260993351.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [90]:
from rich.table import Table
from rich.console import Console

EXPECTED = 34  # expected number of repetitions


def print_repetition_table_rich(df, expected=EXPECTED):
    console = Console()
    table = Table(title="Repetition Completeness per Subject/Visit")

    # Columns
    table.add_column("Subject/Visit", justify="left", style="cyan", no_wrap=True)
    table.add_column("# Present", justify="right", style="green")
    table.add_column("# Expected", justify="right", style="yellow")
    table.add_column("Fraction", justify="center", style="white")
    table.add_column("Percentage", justify="right", style="magenta")

    # Build rows
    for _, row in df.iterrows():
        count_present = int(row["count"])
        fraction = f"{count_present}/{expected}"
        percentage = 100 * count_present / expected

        table.add_row(
            row["subject_visit"],
            str(count_present),
            str(expected),
            fraction,
            f"{percentage:5.1f}%",
        )

    console.print(table)


# Detailed table for repetitions per subject visit
# print_repetition_table_rich(n_subjects_visits, expected=EXPECTED)

In [91]:
from rich.table import Table
from rich.console import Console

EXPECTED = 34  # expected number of repetitions


def print_subject_count_by_repetition(df, expected=EXPECTED):
    console = Console()

    # Aggregate: for each count_present, how many subjects have it?
    agg = (
        df.groupby("count")["subject_visit"]
        .count()
        .rename("n_subjects")
        .reset_index()
        .sort_values("count")
    )

    table = Table(title="Number of Subject/Visit per Repetition Count")

    table.add_column("# Repetitions Present", justify="right", style="cyan")
    table.add_column("# Subject/Visit", justify="right", style="green")
    table.add_column("Fraction of subjects", justify="center", style="white")
    table.add_column("Percentage", justify="right", style="magenta")

    total_subjects = agg["n_subjects"].sum()

    for _, row in agg.iterrows():
        count_present = int(row["count"])
        n_subjects = int(row["n_subjects"])

        frac = f"{n_subjects}/{total_subjects}"
        pct = 100 * n_subjects / total_subjects

        table.add_row(str(count_present), str(n_subjects), frac, f"{pct:5.1f}%")

    console.print(table)


# Example:
print_subject_count_by_repetition(n_subjects_visits, expected=EXPECTED)

                 Number of Subject/Visit per Repetition Count                  
┏━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ # Repetitions Present ┃ # Subject/Visit ┃ Fraction of subjects ┃ Percentage ┃
┡━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│                    25 │               5 │        5/1585        │       0.3% │
│                    26 │              40 │       40/1585        │       2.5% │
│                    27 │              45 │       45/1585        │       2.8% │
│                    28 │             105 │       105/1585       │       6.6% │
│                    29 │             250 │       250/1585       │      15.8% │
│                    30 │             305 │       305/1585       │      19.2% │
│                    31 │             360 │       360/1585       │      22.7% │
│                    32 │             300 │       300/1585       │      18.9% │
│                    33 │             110 │       110/1585       │       6.9% │
│                    34 │              65 │       65/1585        │       4.1% │
└───────────────────────┴─────────────────┴──────────────────────┴────────────┘

## Filter out QC failures 

In [92]:
filename = root_dir / "QC-results" / "qc_results.json"
qc_results = pd.read_json(filename)
qc_results.rename(columns={"subject": "subject_visit"}, inplace=True)
qc_results[["subject", "visit"]] = qc_results["subject_visit"].str.split(
    "_", expand=True
)

### Statistics on QC failures

In [93]:
df_vip_qc = pd.merge(
    df_vip, qc_results, on=["subject_visit", "subject", "visit", "repetition"]
)
df_vip_qc = df_vip_qc.drop(columns=["path", "image_name", "input_dir"])

In [94]:
from typing import Tuple, Callable, Dict, Any
import pandas as pd
from rich.table import Table
from rich.console import Console

console = Console()


# -----------------------------
# Helpers: formatting & colors
# -----------------------------
def _fmt_int(x: Any) -> str:
    try:
        return f"{int(x)}"
    except Exception:
        return str(x)


def _fmt_float(x: Any, ndigits: int = 2) -> str:
    try:
        return f"{float(x):.{ndigits}f}"
    except Exception:
        return str(x)


def _fmt_percent(x: Any, ndigits: int = 2) -> str:
    try:
        return f"{float(x):.{ndigits}f}%"
    except Exception:
        return str(x)


def _percent_color(v: float) -> str:
    """
    Map a percentage value in [0, 100] to a coarse color.
    <25 -> red, <50 -> yellow3, <75 -> chartreuse3, else -> green3
    """
    try:
        p = float(v)
    except Exception:
        return "white"
    if p < 25:
        return "red3"
    elif p < 50:
        return "yellow3"
    elif p < 75:
        return "chartreuse3"
    else:
        return "green3"


def _build_formatters(df: pd.DataFrame) -> Dict[str, Callable[[Any], str]]:
    """
    Default column-wise formatters based on common names/dtypes.
    You can extend/override by passing your own map to show_rich(..., fmt_override=...).
    """
    fmts: Dict[str, Callable[[Any], str]] = {}
    for col in df.columns:
        low = str(col).lower()
        if low in {"percent", "pct", "%"} or low.endswith("%"):
            fmts[col] = _fmt_percent
        elif low.startswith("fraction"):
            fmts[col] = lambda x: _fmt_float(x, 3)  # a bit more precision for fractions
        elif pd.api.types.is_integer_dtype(df[col]):
            fmts[col] = _fmt_int
        elif pd.api.types.is_float_dtype(df[col]):
            fmts[col] = lambda x: _fmt_float(x, 2)
        else:
            # string/object as-is
            fmts[col] = lambda x: f"{x}"
    return fmts


def _colorize_cell(col: str, val_formatted: str, raw_val: Any) -> str:
    """
    Add color tags for specific columns/cases.
    - 'percent' column (by name) gets a color from red->green
    - '#rejected_reps' > 0 is emphasized
    """
    low = str(col).lower()
    if low == "percent" or low.endswith("%"):
        try:
            # Remove trailing '%' for color computation
            p = float(str(val_formatted).rstrip("%"))
        except Exception:
            return val_formatted
        color = _percent_color(p)
        return f"[{color}]{val_formatted}[/]"
    if low in {"#rejected_reps", "#present"}:
        try:
            v = float(raw_val)
            if v > 0 and low == "#rejected_reps":
                return f"[bold red]{val_formatted}[/]"
            elif low == "#present":
                return f"[bold cyan]{val_formatted}[/]"
        except Exception:
            pass
    return val_formatted


# ---------------------------------
# Rich table rendering with styles
# ---------------------------------
def show_rich(
    df: pd.DataFrame,
    title: str,
    fmt_override: Dict[str, Callable[[Any], str]] = None,
    header_style: str = "bold white on grey23",
    row_style: str = "",
):
    """
    Render a pandas DataFrame as a Rich table with:
      - right alignment for numeric columns
      - colored headers
      - colored percentage cells
      - fixed numeric formats (ints/floats/percents)

    Parameters
    ----------
    df : pd.DataFrame
    title : str
    fmt_override : dict (optional)
        Mapping {column_name: formatter_function(value)->str}.
        Overrides the default automatic formatters for those columns.
    header_style : str
        Rich style for column headers.
    row_style : str
        Base style for rows (e.g., 'dim').
    """
    if fmt_override is None:
        fmt_override = {}

    # Build default formatters, then override
    fmts = _build_formatters(df)
    fmts.update(fmt_override)

    # Determine alignment: numeric columns right-aligned
    def _is_numeric_series(s: pd.Series) -> bool:
        return pd.api.types.is_numeric_dtype(s)

    table = Table(
        title=title,
        show_lines=False,
        header_style=header_style,
        row_styles=[row_style] if row_style else None,
    )

    for col in df.columns:
        justify = "right" if _is_numeric_series(df[col]) else "left"
        table.add_column(str(col), justify=justify)

    for _, row in df.iterrows():
        cells = []
        for col in df.columns:
            raw = row[col]
            formatted = fmts[col](raw)
            colored = _colorize_cell(col, formatted, raw)
            cells.append(colored)
        table.add_row(*cells)

    console.print(table)


# =================================
# Rejected image analysis (your fn)
# =================================
def table_rejected(
    df_rejected: pd.DataFrame, expected: int = 34
) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Input columns:
      ['subject_visit', 'repetition', 'subject', 'visit', 'rejected_images']
    Returns:
      per_sv : ['subject_visit', '#rejected_reps', '#expected', 'fraction', 'percent']
      dist   : ['#rejected_reps', 'subjects_with_that_count', 'total_subjects',
                'fraction', 'percent', '#expected']
    """
    # (A) per subject_visit: count repetitions that had any rejection
    per_sv = (
        df_rejected.assign(is_rej=(df_rejected["rejected_images"] > 0).astype(int))
        .groupby("subject_visit", as_index=False)["is_rej"]
        .sum()
        .rename(columns={"is_rej": "#rejected_reps"})
    )
    per_sv["#expected"] = expected
    per_sv["fraction"] = per_sv["#rejected_reps"] / expected
    per_sv["percent"] = per_sv["fraction"] * 100
    per_sv = per_sv.sort_values("subject_visit").reset_index(drop=True)

    # (B) distribution across subject_visit
    dist = (
        per_sv.groupby("#rejected_reps", as_index=False)["subject_visit"]
        .count()
        .rename(columns={"subject_visit": "subjects_with_that_count"})
    )
    total = len(per_sv)
    dist["total_subjects"] = total
    dist["fraction"] = dist["subjects_with_that_count"] / total if total > 0 else 0.0
    dist["percent"] = dist["fraction"] * 100
    dist["#expected"] = expected
    dist = dist.sort_values("#rejected_reps").reset_index(drop=True)

    return per_sv, dist


rej_per_sv, rej_dist = table_rejected(df_vip_qc, expected=34)
# Detailed table for rejected images per subject visit
# show_rich(rej_per_sv, "Per subject_visit: #rejected_reps / #expected and %")
show_rich(rej_dist, "Distribution of #rejected_reps across subject_visits")

                     Distribution of #rejected_reps across subject_visits                      
┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┓
┃ #rejected_reps ┃ subjects_with_that_count ┃ total_subjects ┃ fraction ┃ percent ┃ #expected ┃
┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━┩
│              0 │                      511 │            633 │    0.807 │  80.73% │        34 │
│              1 │                       34 │            633 │    0.054 │   5.37% │        34 │
│              2 │                       15 │            633 │    0.024 │   2.37% │        34 │
│              3 │                       11 │            633 │    0.017 │   1.74% │        34 │
│              4 │                       10 │            633 │    0.016 │   1.58% │        34 │
│              5 │                        8 │            633 │    0.013 │   1.26% │        34 │
│              6 │                        4 │            633 │    0.006 │   0.63% │        34 │
│              7 │                        2 │            633 │    0.003 │   0.32% │        34 │
│              8 │                        3 │            633 │    0.005 │   0.47% │        34 │
│              9 │                        5 │            633 │    0.008 │   0.79% │        34 │
│             10 │                        3 │            633 │    0.005 │   0.47% │        34 │
│             11 │                        4 │            633 │    0.006 │   0.63% │        34 │
│             12 │                        2 │            633 │    0.003 │   0.32% │        34 │
│             13 │                        2 │            633 │    0.003 │   0.32% │        34 │
│             14 │                        2 │            633 │    0.003 │   0.32% │        34 │
│             16 │                        1 │            633 │    0.002 │   0.16% │        34 │
│             18 │                        3 │            633 │    0.005 │   0.47% │        34 │
│             20 │                        1 │            633 │    0.002 │   0.16% │        34 │
│             22 │                        1 │            633 │    0.002 │   0.16% │        34 │
│             25 │                        1 │            633 │    0.002 │   0.16% │        34 │
│             26 │                        1 │            633 │    0.002 │   0.16% │        34 │
│             28 │                        4 │            633 │    0.006 │   0.63% │        34 │
│             29 │                        1 │            633 │    0.002 │   0.16% │        34 │
│             30 │                        1 │            633 │    0.002 │   0.16% │        34 │
│             31 │                        1 │            633 │    0.002 │   0.16% │        34 │
│             32 │                        1 │            633 │    0.002 │   0.16% │        34 │
│             34 │                        1 │            633 │    0.002 │   0.16% │        34 │
└────────────────┴──────────────────────────┴────────────────┴──────────┴─────────┴───────────┘

In [95]:
from typing import Tuple, Callable, Optional, Dict, Any
import pandas as pd
from rich.table import Table
from rich.console import Console
from rich import box

console = Console()


# ---------- formatting helpers ----------
def _default_formatters(df: pd.DataFrame) -> Dict[str, Callable[[Any], str]]:
    """Default number formatting based on column names/dtypes."""
    fmts: Dict[str, Callable[[Any], str]] = {}
    int_like_cols = [c for c in df.columns if c.startswith("#")]
    for col in df.columns:
        if col in int_like_cols or pd.api.types.is_integer_dtype(df[col]):
            fmts[col] = lambda x: f"{int(x):,}" if pd.notna(x) else ""
        elif col.lower() == "fraction":
            fmts[col] = lambda x: f"{float(x):.3f}" if pd.notna(x) else ""
        elif col.lower() == "percent":
            fmts[col] = lambda x: f"{float(x):.2f}%" if pd.notna(x) else ""
        else:
            fmts[col] = lambda x: str(x) if pd.notna(x) else ""
    return fmts


def show_rich(
    df: pd.DataFrame,
    title: str,
    fmt: Optional[Dict[str, Callable[[Any], str]]] = None,
    row_style_fn: Optional[Callable[[pd.Series], Optional[str]]] = None,
    box_style=box.MINIMAL_HEAVY_HEAD,
    header_style: str = "bold cyan",
    caption: Optional[str] = None,
) -> None:
    """Render a Rich table with formatting and optional per-row color."""
    if fmt is None:
        fmt = _default_formatters(df)
    table = Table(
        title=title, show_lines=False, box=box_style, header_style=header_style
    )
    for col in df.columns:
        table.add_column(str(col))
    for _, row in df.iterrows():
        style = row_style_fn(row) if row_style_fn else None
        cells = [fmt.get(col, lambda x: str(x))(row[col]) for col in df.columns]
        table.add_row(*cells, style=style)
    console.print(table)
    if caption:
        console.print(caption)


def subjects_threshold_table(
    df_present: pd.DataFrame, threshold: int = 26
) -> pd.DataFrame:
    """
    Returns a 2-row table counting subject_visits with #present < threshold vs ≥ threshold.
    df_present columns: ['subject_visit','count']
    """
    total = len(df_present)
    less = int((df_present["count"] < threshold).sum())
    geq = int(total - less)
    out = pd.DataFrame(
        {
            "category": [f"< {threshold}", f"≥ {threshold}"],
            "subjects": [less, geq],
            "total_subjects": [total, total],
            "fraction": [
                less / total if total > 0 else 0.0,
                geq / total if total > 0 else 0.0,
            ],
            "percent": [0.0, 0.0],
        }
    )
    out["percent"] = out["fraction"] * 100.0
    return out


# ---------- row style helpers ----------
def style_by_threshold_percent(
    row: pd.Series, threshold_percent: float
) -> Optional[str]:
    """Color a row red if percent<threshold, green otherwise."""
    val = row.get("percent", None)
    if pd.isna(val):
        return None
    return "bold red" if float(val) < threshold_percent else "bold green"


def style_for_lt_ge_table(row: pd.Series) -> Optional[str]:
    """Color '< threshold' red and '≥ threshold' green."""
    cat = str(row.get("category", ""))
    if cat.startswith("<"):
        return "bold red"
    if cat.startswith("≥"):
        return "bold green"
    return None

### Filter out failing subject/visit

In [96]:
df_vip_qc_success = df_vip_qc[~df_vip_qc["rejected_images"]]
df_present = (
    df_vip_qc_success.groupby("subject_visit", as_index=False)["repetition"]
    .count()
    .rename(columns={"repetition": "count"})
)
lt_ge_tbl = subjects_threshold_table(df_present, threshold=26)
show_rich(
    lt_ge_tbl,
    "Subjects by success threshold (<26 vs ≥26)",
    row_style_fn=style_for_lt_ge_table,
)

         Subjects by success threshold (<26 vs ≥26)          
           ╷          ╷                ╷          ╷          
  category │ subjects │ total_subjects │ fraction │ percent  
╺━━━━━━━━━━┿━━━━━━━━━━┿━━━━━━━━━━━━━━━━┿━━━━━━━━━━┿━━━━━━━━━╸
  < 26     │ 48       │ 627            │ 0.077    │ 7.66%    
  ≥ 26     │ 579      │ 627            │ 0.923    │ 92.34%   
           ╵          ╵                ╵          ╵

### Filter out subject without 2 visits per repetition

In [134]:
df_vip_qc_success_2_visits = df_vip_qc_success.copy()
# pivot again, but inspect column names
pairs = df_vip_qc_success_2_visits.pivot_table(
    index=["subject", "repetition"],
    columns="visit",
    values="subject_visit",
    aggfunc="first",
    observed=True,
).reset_index()
df_vip_qc_success_2_visits = pairs[
    (~pairs[pairs.columns[2:]].isnull()).sum(axis=1) == 2
]
sessions_labels = [c for c in df_vip_qc_success_2_visits.columns if "ses-" in c]
df_vip_qc_success_2_visits = df_vip_qc_success_2_visits.melt(
    id_vars=["subject", "repetition"],
    var_name="visit",
    value_name="subject_visit",
    value_vars=sessions_labels,
)

df_vip_qc_success_2_visits = pd.merge(
    df_vip_qc_success,
    df_vip_qc_success_2_visits,
    left_on=["subject", "repetition", "visit", "subject_visit"],
    right_on=["subject", "repetition", "visit", "subject_visit"],
)
df_present = (
    df_vip_qc_success_2_visits.groupby("subject_visit", as_index=False)["repetition"]
    .count()
    .rename(columns={"repetition": "count"})
)
lt_ge_tbl = subjects_threshold_table(df_present, threshold=26)
show_rich(
    lt_ge_tbl,
    "Subjects by success threshold (<26 vs ≥26)",
    row_style_fn=style_for_lt_ge_table,
)

         Subjects by success threshold (<26 vs ≥26)          
           ╷          ╷                ╷          ╷          
  category │ subjects │ total_subjects │ fraction │ percent  
╺━━━━━━━━━━┿━━━━━━━━━━┿━━━━━━━━━━━━━━━━┿━━━━━━━━━━┿━━━━━━━━━╸
  < 26     │ 88       │ 622            │ 0.141    │ 14.15%   
  ≥ 26     │ 534      │ 622            │ 0.859    │ 85.85%   
           ╵          ╵                ╵          ╵

### Final QCed cohort

In [ ]:
df_present_ge_26 = df_present[df_present["count"] >= 26]
df_vip_qc_success_2_visits_ge_26 = df_vip_qc_success_2_visits[
    df_vip_qc_success_2_visits["subject_visit"].isin(df_present_ge_26["subject_visit"])
]
df_vip_qc_n_visits = df_vip_qc_success_2_visits_ge_26.groupby("subject")[
    "visit"
].nunique()
df_vip_qc_1_visit = df_vip_qc_n_visits[df_vip_qc_n_visits == 1]
df_vip_qc_2_visits = df_vip_qc_n_visits[df_vip_qc_n_visits == 2]
print(f"Number of subjects with 1 visit passed QC: {len(df_vip_qc_1_visit)}")
print(f"Number of subjects with 2 visits passed QC: {len(df_vip_qc_2_visits)}")

df_vip_qc_success_2_visits_ge_26 = df_vip_qc_success_2_visits_ge_26[
    df_vip_qc_success_2_visits_ge_26["subject"].isin(df_vip_qc_2_visits.index)
]

# Assert 26 repetitions per subject/visit
assert all(
    df_vip_qc_success_2_visits_ge_26.groupby(["subject_visit"])["repetition"].nunique()
    >= 26
)

n_subjects = df_vip_qc_success_2_visits_ge_26["subject"].nunique()
print(
    f"Total number of subjects passed QC with ≥26 successful repetitions: {n_subjects}"
)
filename = (
    output_dir / "vip_executions_stats_info_2visits_passed_qc_with_26_repetitions.csv"
)
df_vip_qc_success_2_visits_ge_26.to_csv(filename, index=False)
print(f"Saved VIP executions stats info (success ≥26) to {anondir(filename)}")

Number of subjects with 1 visit passed QC: 0
Number of subjects with 2 visits passed QC: 267
Total number of subjects passed QC with ≥26 successful repetitions: 267
Saved VIP executions stats info (success ≥26) to <living-park>/cohort/vip_executions_stats_info_2visits_passed_qc_with_26_repetitions.csv
